In [1]:
%pip install fastapi pydantic toml openai

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check, chat_with_azure
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|medium|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat_with_azure(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    content = f"I am making a skill check using {skill.upper()} against a difficulty of {difficulty.upper()}."

    d = difficulty.lower()
    dc = 50
    if d == "easy":
        dc = 75
    elif d == "hard":
        dc = 25
    roll = d20.roll("1d100").total
    result = "Success" if roll <= dc else "Failure"
    if roll == 1:
        result = "Critical Success"
    elif roll == 100:
        result = "Critical Failure"

    content += f"\nAnd I rolled a {roll} for a result of {result.upper()}. {get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "actions" in dict.keys():
        matches = action_pattern.finditer(dict["actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        difficulty = dict["difficulty"]
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add round summary as a memory to the metadata
        # it will be used in the next round.
        # config.metadata["memories"] = []
        if ("memories" not in config.metadata):
            config.metadata["memories"] = []
            
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    elif "ending" in dict.keys():
        ending_btn = widgets.Button(description="END", layout=widgets.Layout(width="auto"))
        buttons.append(ending_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/coc_lot_66_4.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round =0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))


context: You arrive at an old self-storage facility by invitation from the manager, who has an storage room up for auction due to the death of its owner. As you and other interested buyers gather around the auctioneer, a worker carrying a bolt cutter snaps off the rusty lock, unveiling a storage room cluttered with various old furniture and antiquities. Without giving any opportunity for the bidders to take a clear look, the auctioneer begins the bid at $50.
What's your bid?
actions:
1. Bid $100
2. Bid $150
3. Bid $200


I select option 2. TURN: 1 


context: You raise your hand, bidding $150. The auctioneer acknowledges your bid with a nod. Almost immediately, another bidder, a well-dressed woman with a sharp gaze, raises the bid to $200. The room is filled with tension as eyes dart between the two of you, waiting for the next move.
actions:
1. Bid $250
2. Bid $300
3. Stay at $150

I select option 1. TURN: 2 


context: With a determined look, you bid $250. The woman hesitates for a moment but counters with a bid of $300. The auctioneer's voice rises in excitement, sensing the competition heating up. The other bidders seem to back off, leaving the contest between you and the woman.
actions:
1. Bid $350
2. Bid $400
3. Stay at $250

I select option 1. TURN: 3 


context: You bid $350, pushing the stakes higher. The woman pauses, her expression unreadable, then she surprisingly bows out with a subtle shake of her head. The auctioneer calls out for any more bids, but the room remains silent, all eyes on you.
skill: Persuasion
difficulty: normal

I am making a skill check using PERSUASION against a difficulty of NORMAL.
And I rolled a 20 for a result of SUCCESS. TURN: 4 


context: Your confident demeanor and decisive bidding seem to have dissuaded others from challenging you further. The auctioneer, after a brief pause, declares you the winner of the auction at $350. Applause breaks out among the bystanders as you approach the manager to settle the payment and collect the keys to your newly acquired storage room.
summary: Winning the storage room auction for $350 after a tense bidding war, you've managed to secure the contents of the mysterious storage room. Your successful persuasion ensured no further bids against you, leaving you victorious. With $100 left in your pocket, you're now the owner of items whose secrets and values are yet to be discovered.

...

I select option 1. TURN: 1 


context: You sprint towards the exit, adrenaline pumping through your veins. The eerie sound of the withered body's movements echoes behind you, a chilling reminder of the horror that's on your tail. As you reach the exit, you find it unexpectedly locked, the door unyielding to your desperate tugs.

skill: Athletics
difficulty: hard

I am making a skill check using ATHLETICS against a difficulty of HARD.
And I rolled a 80 for a result of FAILURE. TURN: 2 


context: Your attempts to force the door open fail, leaving you gasping for breath, your muscles aching from the exertion. The relentless pursuit of the

look for a weapon. TURN: 3 


context: Frantically searching your surroundings, you spot a heavy metal pipe lying near some discarded boxes. It's not much, but it's better than facing the creature unarmed. The withered body's grotesque form is now ominously close, its movements a dreadful symphony of unnatural sounds.

actions:
1. Prepare to fight with the metal pipe.
2. Continue trying to open the door.
3. Attempt to communicate with the creature.

I select option 1. TURN: 4 


context: Gripping the metal pipe with determined resolve, you turn to face the nightmarish entity. Its form is a blur of shadows and malice, a testament to the dark rituals that gave it life. The air grows cold, and the very atmosphere seems to thicken with impending doom as it approaches.

skill: Fighting (Brawl)
difficulty: extreme

I am making a skill check using FIGHTING against a difficulty of EXTREME.
And I rolled a 72 for a result of FAILURE. TURN: 5 


context: Your swing with the pipe passes through the air, missing the entity as it deftly sidesteps. The creature's counter is swift, a mass of tentacles ensnaring the pipe and wrenching it from your grasp with supernatural strength. You're left defenseless, staring into the abyss of its form, as it looms over you, ready to strike.

summary: In a desperate attempt to escape the storage facility, you ran towards the only exit, only to find it locked. A failed attempt to force it open left you vulnerable. Seeking a weapon, you armed yourself with a metal pipe, but your attempt to fight the monstrous entity resulted in failure. Disarmed and overpowered, your final stand against the darkness was futile. The withered body, a relic of forbidden rituals and demonic summoning, claimed you, its wrath unquenched.